## Setup 

In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("Delta Lake Exploration") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/sharattadimalla/github/deltalake/myenv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/sharattadimalla/.ivy2/cache
The jars for the packages stored in: /Users/sharattadimalla/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-848917e5-40c0-484f-96f6-8bc9a2dbbcdc;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.1 in central
	found io.delta#delta-storage;2.1.1 in central
	found org.antlr#antlr4-runtime;4.8 in local-m2-cache
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in local-m2-cache
:: resolution report :: resolve 258ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.1 from central in [default]
	io.delta#delta-storage;2.1.1 from central in [default]
	org.antlr#antlr4-runtime;4.8 from local-m2-cache in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |

22/11/27 11:49:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## create a sample dataframe

In [4]:
df = spark.range(0,5)
df.printSchema()
df.schema

root
 |-- id: long (nullable = false)



StructType([StructField('id', LongType(), False)])

In [5]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



## Create a delta table

## Insert records into delta table

In [9]:
df.write.format("delta").mode("overwrite").save("output/delta/sample_tbl")

In [13]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import col

new_df = df.select(col("id").cast(StringType()).alias("id_str"))
new_df.schema

StructType([StructField('id_str', StringType(), False)])

## Schema Validation - write string value to int column 

In [14]:
new_df.write.format("delta").mode("append").save("output/delta/sample_tbl")

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 0df055e7-465d-4022-bfdc-4d35c2752cff).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- id: long (nullable = true)


Data schema:
root
-- id_str: string (nullable = true)

         

## Schema Validation Exception

```
AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 0df055e7-465d-4022-bfdc-4d35c2752cff).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- id: long (nullable = true)


Data schema:
root
-- id_str: string (nullable = true)
```